### 1. Tạo pertured data và lưu giống data cũ (cal_metric.py)
    1. Tạo ra câu pertured va convert json và ghép label (pertured_folder/masked_data_json)
### 2. Lấy ra vector logit 
    1. convert interim -> txt 
    2. transform, prepare -> json 
    3. đưa json (masked data 0.1) vào model lấy logit, prediction -> cần tạo class để lấy cho cả origin data và masked data
### 3. convert label gold theo từ thành label gold theo token
### 4. Tính inf 

## Step 2

In [ ]:
# 2.1 Done
# convert csv to coNLL format and write to txt file
!python ../data_transformations.py --transform_file '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/SRL/transform_file_mlm.yml'
# output file MLM/coNLL_txt

In [ ]:
# 2.2 Done
!python ../data_transformations.py --transform_file '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/SRL/transform_file_conll.yml'
# nhớ chỉnh lại đường dẫn trong file transform_file_conll.yml

In [ ]:
# 2.2 Done
# !python ../data_preparation.py --task_file tasks_file_SRL.yml --data_dir ../data/coNLL_tsv --max_seq_len 50

!python ../data_preparation.py \
        --task_file "/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/SRL/tasks_file_SRL.yml" \
        --data_dir ../MLM/coNLL_tsv_json \
        --max_seq_len 85

## Calculation influence, relevance and competence score

In [4]:

!python metrics_calculation.py \
    --data_mask_dir './data_mlm/perturbed_data/avg_neg_cos/' \
    --data_origin_dir './data_mlm/process_folder/coNLL_tsv_json/ner_json/'  \
    --model_path '../output/multi_task_model_9_13050.pt' \
    --log_name "cal_corr_inf_lhs" 


/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2024-06-12 14:36:47.433226: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-12 14:36:47.478601: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-12 14:36:48.166853: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-06-12 14:36:48.969472: E tensorflow/compiler/xla/stream_executor/cuda/cu

## calculate metrics for replace mask token

In [ ]:
!python metrics_calculation.py \
    --data_mask_dir './data_mlm/process_folder/mlm_output/' \
    --data_origin_dir './data_mlm/process_folder/coNLL_tsv_json/ner_json/'  \
    --model_path '../output/multi_task_model_9_13050.pt' \
    --log_name "comp_mask_token" \
    --is_mask_token True

## calculate metrics for delete mask token

In [ ]:
!python metrics_calculation.py \
    --data_mask_dir './data_mlm/process_folder/mlm_output/' \
    --data_origin_dir './data_mlm/process_folder/coNLL_tsv_json/ner_json/'  \
    --model_path '../output/multi_task_model_9_13050.pt' \
    --log_name "comp_del_mask_token" \
    --del_mask_token True

## Task 1.1 :

In [25]:
from datetime import datetime
import os
from mlm_utils.transform_func import get_files
from metrics_calculation import get_pair_inf_rel
from data_maker import DataMaker
from infer_pipeline import inferPipeline
from logger_ import make_logger
def get_word(dataDir, file_name, model, labelRn, wrtDir=None, hasTrueLabels=True, needMetrics=True, is_mask_token=False, del_mask_token=False):
    dataMaker = DataMaker(
        data_file= os.path.join(dataDir, file_name)
    )
    
    result = dataMaker.evaluate(model, labelRn, wrtPredPath=file_name, wrtDir=wrtDir, returnPreds=True, hasTrueLabels=hasTrueLabels, needMetrics=needMetrics, is_mask_token=is_mask_token, del_mask_token=del_mask_token)
    return {
        'uid': result[0],
        'pred': result[1],
        'score': result[2],
        'logitsSoftmax': result[3],
        'logitsRaw': result[4],
        'label': result[5] # masked word kh cos label
    }
    
dataMaskedDir = './data_mlm/perturbed_data/avg_pos_cos_module/'
dataOriginDir = './data_mlm/process_folder/coNLL_tsv_json/ner_json/'
# mask = 'abolish.json' 
# origin = 'ner_conll_format_abolish_full.json'
model_path = '../output/multi_task_model_9_13050.pt'
# setting logging
now = datetime.now()
logDir = now.strftime("%d_%m-%H_%M")
if not os.path.isdir(logDir):
    os.makedirs(logDir)

logger = make_logger(name = 'test_corr_inf_lhs', debugMode=True,
                    logFile=os.path.join(logDir, '{}.log'.format('test_corr_inf_lhs')), silent=True)
logger.info("logger created.")


pipe = inferPipeline(logger, model_path)
labelMap = pipe.taskParams.labelMap['conllsrl']
labelRn = {v:k for k,v in labelMap.items()}
 
file_mask = sorted(get_files(dataMaskedDir))
file_origin = sorted(get_files(dataOriginDir))
all_inf_score = []

for mask, origin in zip(file_mask, file_origin):   
    resultwordMasked = get_word(dataMaskedDir, mask, pipe.model, labelRn, hasTrueLabels=False, needMetrics=False, is_mask_token=False, del_mask_token=False)
    resultOrigin = get_word(dataOriginDir, origin, pipe.model, labelRn, hasTrueLabels=True, needMetrics=False)

    # labelMap = {v: k for k, v in labelRn.items()}
    comp_score, list_inf_score = get_pair_inf_rel(resultOrigin, resultwordMasked, labelMap)
    all_inf_score.append(list_inf_score)

 67%|██████▋   | 132/198 [08:26<04:38,  4.22s/it]

In [ ]:
len(all_inf_score)

1022

In [ ]:
# read csv 
import pandas as pd

cosin_dir = './data_mlm/process_folder/cosine_similarity'

files = get_files(cosin_dir)

all_cosine = pd.DataFrame()
for file in files:
    cos = pd.read_csv(os.path.join(cosin_dir, file))
    all_cosine = pd.concat([all_cosine, cos], axis=0, ignore_index = True)


,cos_sim,cos_mo,cos_dif,cos_mo_dif,ele_sub
0,0.00674,0.00590,0.99326,0.99410,0.00934
1,-0.06479,-0.05743,0.93521,0.94257,0.01097
2,0.02651,0.02541,0.97349,0.97459,0.03559
3,-0.00867,-0.00758,0.99133,0.99242,0.02029
4,-0.09000,-0.07904,0.91000,0.92096,0.01182


In [ ]:
len(all_cosine['cos_sim'].tolist())

1024

In [24]:
# cal spearman correlation between list_inf_score and cos['cos_sim']
from scipy.stats import spearmanr
correlation_coefficient, p_value = spearmanr(list_inf_score, cos['cos_mo_dif'][:-2].tolist())
print(correlation_coefficient, p_value) 

-0.05535127790642139 0.07694380682028741
